In [14]:
# Import necessary libraries
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import PIL.Image as Image
import pathlib
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [15]:
# Get directory path to Training dataset
train_dir = pathlib.Path('../input/gender-recognition-dataset/Gender Recognition Dataset/Training')
# Get a list of all images in the Training dataset
train_image_paths = list(train_dir.glob(r'**/*.jpg'))

# Get directory path to Validation dataset
valid_dir = pathlib.Path('../input/gender-recognition-dataset/Gender Recognition Dataset/Validation')
# Get a list of all images in the Validation dataset
valid_image_paths = list(valid_dir.glob(r'**/*.jpg'))

## Image Processing

In [16]:
# Create a function to extract the labels from image filepath
def image_processing(filepath):
    labels = [str(filepath[i]).split('/')[-2]
             for i in range(len(filepath))]
    
    # Create a DataFrame and input the filepath and labels
    filepath = pd.Series(filepath, name = 'Filepath').astype(str)
    labels = pd.Series(labels, name = 'Label')
    
    df = pd.concat([filepath, labels], axis='columns')
    
    return df

In [17]:
# Create a train and validation DataFrame
train_df = image_processing(train_image_paths)
val_df = image_processing(valid_image_paths)

In [18]:
# Create DataFrame with just one label for each label
df_unique = train_df.copy().drop_duplicates(subset=['Label']).reset_index()

In [19]:
# Generate new images from dataset
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function = tf.keras.applications.mobilenet_v3.preprocess_input
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
preprocessing_function = tf.keras.applications.mobilenet_v3.preprocess_input
)

In [20]:
# Generate images using 'train_df' DataFrame
train_images = train_generator.flow_from_dataframe(
    dataframe  = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 0,
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

Found 47406 validated image filenames belonging to 2 classes.


In [21]:
# Generate images using 'val_df' DataFrame
val_images = train_generator.flow_from_dataframe(
    dataframe  = val_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed = 0,
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

Found 11831 validated image filenames belonging to 2 classes.


In [22]:
# Use Tensorflow pretrained model
pretrained_model = tf.keras.applications.MobileNetV3Small(
input_shape= (224, 224, 3),
include_top = False,
weights = 'imagenet',
pooling = 'avg'
)

# Freeze weights
pretrained_model.trainable = False

In [23]:
# Create weights
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation = 'relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)

outputs = tf.keras.layers.Dense(2, activation = 'softmax')(x)

model = tf.keras.Model(inputs = inputs, outputs = outputs)

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [24]:
# Train model
history = model.fit(
    train_images,
    validation_data = val_images,
    batch_size = 32,
    epochs = 20,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = 2,
            restore_best_weights = True
        )  
    ]
)

Epoch 1/20
1482/1482 [==============================] - 230s 154ms/step - loss: 0.2220 - accuracy: 0.9127 - val_loss: 0.1786 - val_accuracy: 0.9291
Epoch 2/20
1482/1482 [==============================] - 106s 71ms/step - loss: 0.1816 - accuracy: 0.9307 - val_loss: 0.1777 - val_accuracy: 0.9284
Epoch 3/20
1482/1482 [==============================] - 106s 72ms/step - loss: 0.1681 - accuracy: 0.9358 - val_loss: 0.1731 - val_accuracy: 0.9320
Epoch 4/20
1482/1482 [==============================] - 101s 68ms/step - loss: 0.1588 - accuracy: 0.9405 - val_loss: 0.1688 - val_accuracy: 0.9340
Epoch 5/20
1482/1482 [==============================] - 106s 71ms/step - loss: 0.1514 - accuracy: 0.9430 - val_loss: 0.1635 - val_accuracy: 0.9386
Epoch 6/20
1482/1482 [==============================] - 104s 70ms/step - loss: 0.1444 - accuracy: 0.9460 - val_loss: 0.1629 - val_accuracy: 0.9379
Epoch 7/20
1482/1482 [==============================] - 106s 71ms/step - loss: 0.1369 - accuracy: 0.9495 - val_loss: 

In [25]:
# Create labels dictionary
labels = {0: 'female',
         1: 'male'}

In [26]:
# Create a function for image processing and prediction
def output(imagepath):
    img = image.load_img(imagepath, target_size=(224, 224, 3))
    img = img_to_array(img)
    img = img/255
    img = np.expand_dims(img, [0])
    
    answer = model.predict(img)[0]
    
    idx = answer.argmax()
    res = labels[idx]
    
    return res

In [27]:
# Predict gender
img = output('../input/gender-classification-dataset/Validation/female/112953.jpg.jpg')
img

'female'

In [28]:
# Save model
model.save('GR.h5')

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
